# validate_claims.ipynb
Cross-checks LLM statements in `results/responses.csv` against ground truth in `analysis/ground_truth.json`.

**Output:**
- `analysis/fabrication_report.json` — per-hypothesis contradiction counts and an estimated fabrication rate.

**What it checks (rule-based):**
- Mentions of games played vs actual `games_played`.
- Statements implying 4th-period goals are higher than 1st when ground truth says otherwise.

You can extend the rules for additional claims if needed.

In [4]:
import csv, json, re
from pathlib import Path
from collections import defaultdict

ROOT = Path('.')
RESP = ROOT / 'results' / 'responses.csv'
GT = ROOT / 'analysis' / 'ground_truth.json'
OUT = ROOT / 'analysis' / 'fabrication_report.json'

print('Paths:')
print(' - responses:', RESP)
print(' - ground_truth:', GT)
print(' - output:', OUT)

Paths:
 - responses: results\responses.csv
 - ground_truth: analysis\ground_truth.json
 - output: analysis\fabrication_report.json


In [5]:
if not RESP.exists():
    raise SystemExit('No responses.csv found. Run run_experiment to log outputs first.')
if not GT.exists():
    raise SystemExit('Missing analysis/ground_truth.json. Generate or move your ground truth file into analysis/.')

gt = json.loads(GT.read_text())
games = gt.get('games_played')
p1 = (gt.get('period_goals') or {}).get('P1')
p4 = (gt.get('period_goals') or {}).get('P4')

# Simple regex detectors for numbers mentioned
num_pattern = re.compile(r"\b(\d{1,3})\b")

contradiction_counts = defaultdict(int)
total_counts = defaultdict(int)

with RESP.open(encoding='utf-8') as f:
    for row in csv.DictReader(f):
        h = row.get('hypothesis_id','')
        text = row.get('response_text','')
        total_counts[h] += 1

        low = text.lower()

        # Check games played if mentioned
        if 'game' in low and games is not None:
            for m in num_pattern.findall(text):
                try:
                    n = int(m)
                except:
                    continue
                # Heuristic: numbers in a plausible season range
                if 5 <= n <= 40 and n != games:
                    contradiction_counts[h] += 1
                    break

        # Check period goals trend if mentioned
        if (('first period' in low or '1st period' in low or 'p1' in low) and 
            ('fourth period' in low or '4th period' in low or 'p4' in low)):
            if p1 is not None and p4 is not None:
                if ('higher in fourth' in low or 'improved in fourth' in low or 'more goals in fourth' in low):
                    if p4 < p1:
                        contradiction_counts[h] += 1

fabrication_rate = {h: (contradiction_counts[h]/total if total>0 else 0.0)
                    for h,total in total_counts.items()}

report = {
    'fabrication_rate_by_hypothesis': fabrication_rate,
    'contradictions_by_hypothesis': dict(contradiction_counts),
    'total_responses_by_hypothesis': dict(total_counts)
}

OUT.write_text(json.dumps(report, indent=2))
print(f'Wrote {OUT}')

Wrote analysis\fabrication_report.json


In [6]:
# Quick preview of the output JSON (if created)
from pathlib import Path
out = Path('analysis/fabrication_report.json')
if out.exists():
    txt = out.read_text()
    print(txt[:1000] + ('...' if len(txt)>1000 else ''))
else:
    print('fabrication_report.json not found yet.')

{
  "fabrication_rate_by_hypothesis": {
    "": 0.0,
    "H1": 0.0,
    "H2": 0.0,
    "H3": 0.0,
    "H4": 0.0
  },
  "contradictions_by_hypothesis": {
    "": 0,
    "H1": 0,
    "H2": 0,
    "H3": 0,
    "H4": 0
  },
  "total_responses_by_hypothesis": {
    "": 1,
    "H1": 4,
    "H2": 1,
    "H3": 2,
    "H4": 2
  }
}
